In speech processing we may use really huge networks cause majority of the processing bottleneck is on CPU and running the CTC function...

Also, CLARIN mobile corpus seems to be too small for advanced network

In [1]:
ls datasets/clarin-long/data

SES0001/  SES0124/  SES0247/  SES0371/  SES0494/
SES0002/  SES0125/  SES0248/  SES0372/  SES0495/
SES0003/  SES0126/  SES0249/  SES0373/  SES0496/
SES0004/  SES0127/  SES0250/  SES0374/  SES0497/
SES0005/  SES0128/  SES0251/  SES0375/  SES0498/
SES0006/  SES0129/  SES0252/  SES0376/  SES0499/
SES0007/  SES0130/  SES0253/  SES0377/  SES0500/
SES0008/  SES0131/  SES0254/  SES0378/  SES0501/
SES0009/  SES0132/  SES0255/  SES0379/  SES0502/
SES0010/  SES0133/  SES0256/  SES0380/  SES0503/
SES0011/  SES0134/  SES0257/  SES0381/  SES0504/
SES0012/  SES0135/  SES0258/  SES0382/  SES0505/
SES0013/  SES0136/  SES0259/  SES0383/  SES0506/
SES0014/  SES0137/  SES0260/  SES0384/  SES0507/
SES0015/  SES0138/  SES0261/  SES0385/  SES0508/
SES0016/  SES0139/  SES0262/  SES0386/  SES0509/
SES0017/  SES0140/  SES0263/  SES0387/  SES0510/
SES0018/  SES0141/  SES0264/  SES0388/  SES0511/
SES0019/  SES0142/  SES0265/  SES0389/  SES0512/
SES0020/  SES0143/  SES0266/  SES0390/  SES0513/
SES0021/  SES0144/  

In [4]:
#!/usr/bin/python3

# For demonstration purposes - Paweł Tomasik

# for CLARIN_MOBILE - generally it is unnormalized

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from keras.models import Model
from keras.layers import LSTM, Conv1D, Dropout, LeakyReLU, Dense, Input, Lambda, TimeDistributed, Flatten, Conv2D, BatchNormalization, GRU
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import Orthogonal

from keras.backend import ctc_batch_cost, expand_dims
import keras.backend as K

import editdistance  # For digit error rate
import keras
import librosa
import numpy as np
import os

LENGTH = 2648
RECS = 1384 * 9
TRANSL = 307
PHONES = 36

X = np.zeros([RECS, LENGTH, 20], np.float32)
Y = np.zeros([RECS, LENGTH // 2, PHONES], np.float32)
counter = 0

for i in range(9):
    Xpart = np.load("datasets/clarin-long/data/clarin-mfcc-rec-aligned-{}.npy".format(i))
    Ypart = np.load("datasets/clarin-long/data/clarin-mfcc-trans-aligned-{}.npy".format(i))
    recs = Xpart.shape[0]
    reclen = Xpart.shape[1]
    translen = Ypart.shape[1]
    X[counter : counter + recs, :reclen, :] = Xpart
    Y[counter : counter + recs, :translen] = Ypart
    counter += recs
    
print(counter, RECS)
counter //= 32
counter *= 32

X = X[:counter]
Y = Y[:counter]

#X = np.clip(X, -3, 3)
MEAN = X.mean()
STD = X.std()


# dodać szum
# przejrzeć wagi, callback
# funkcja krosentropii
# uwzględnić padding, więc powinniśmy uzupełnić funkcję straty
# padding do końca sekwencji, nie do początku


12429 12456


In [ ]:
NFEATS = 20

def mk_model(max_label_length):
    feature_input = Input(shape = (None, NFEATS))
    layer = Lambda(lambda x: (x - MEAN) / STD)(feature_input)
    layer = Lambda(K.expand_dims)(layer)
    layer_1 = Conv2D(12, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer_1 = Conv2D(16, [5,1], activation='linear', kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer = TimeDistributed(Flatten())(layer)
    layer_10 = LSTM(64, return_sequences = True, kernel_initializer=Orthogonal(), kernel_regularizer=keras.regularizers.l1(1e-7), bias_regularizer=keras.regularizers.l1(1e-6))(layer)
    layer_11 = LSTM(64, return_sequences = True, kernel_initializer=Orthogonal(), kernel_regularizer=keras.regularizers.l1(1e-7), bias_regularizer=keras.regularizers.l1(1e-6))(layer_10)
    layer_14 = LSTM(PHONES, return_sequences = True, activation = 'linear', kernel_regularizer=keras.regularizers.l1(1e-6), bias_regularizer=keras.regularizers.l1(1e-6))(layer_11)
    layer_15 = LeakyReLU(0.01)(layer_14)
    model = predictive = Model(feature_input, layer_15)
    model.summary()
    return model, model

def train(model, trainX, trainy, epochs = 50):
    # important: batch_size=1 bugs Tensorflow
    optimizer = Adam(0.01, clipnorm=1.)
    model.compile(loss='mse', optimizer=optimizer)
    return model.fit(trainX, trainy, epochs = epochs, batch_size = 32)

def validate(predictions, valid_length, groundtruth, target_length):
    predictions = keras.backend.ctc_decode(predictions, valid_length, False, 1)
    predictions = predictions[0][0].eval(session=keras.backend.get_session())
    DERs = []
    for index in range(predictions.shape[0]):
        dist = float(editdistance.eval(
            [x for x in predictions[index, :] if x != -1],
            [x for x in groundtruth[index, :] if x != NPHONES]))
        DER = dist / target_length[index]
        DERs.append((DER, target_length[index]))
    return DERs

def try_else(exp, exp_else):
    try:
        return exp()
    except:
        return exp_else

if __name__=='__main__':
    data = X, Y
    trn, predict = mk_model(data[1].shape[1])
    train(trn, *data, epochs=1000) # at 300 it makes sensible predictions
    valid_data = make_data(*load_data(VALIDDATA))
    predictions = predict.predict(valid_data[0])
    DERs = validate(predictions, valid_data[2], valid_data[1], valid_data[3])
    print("Validation Digit Error Rate: {}".format(sum([x[0] * x[1] for x in DERs]) / sum([x[1] for x in DERs])))
    predict.save('cyfry.pred.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 20)          0         
_________________________________________________________________
lambda_1 (Lambda)            (None, None, 20)          0         
_________________________________________________________________
lambda_2 (Lambda)            (None, None, 20, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, 20, 12)      72        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, None, 20, 12)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 20, 12)      48        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, 20, 16)      976       
__________

In [13]:
GRU(np.zeros(NPHONES + 1))(Input((30,10)))

TypeError: only size-1 arrays can be converted to Python scalars

In [17]:
pr = predict.predict(X[:1])
pr = pr.argmax(2)[0]
pr

array([20, 20, 20, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 11, 37,  6, 37, 37, 28, 28, 37, 37,  4, 37, 29, 37, 37, 28, 37,
       37, 37, 37, 31, 37, 37, 37, 37, 37, 37, 37, 36, 37,  6, 37, 37, 36,
       37,  8, 37,  5, 37, 37, 37, 37,  7, 37, 33, 37, 37, 37, 29, 29, 37,
        1, 37,  8, 37, 37, 37, 37, 37, 37, 37, 37, 37,  0,  0,  6, 37, 37,
       37,  4, 37, 33, 37, 37, 37,  0,  0, 37, 37, 37,  6,  6, 37, 37, 37,
       20, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 31, 31, 37,
       28, 28, 37, 37, 37, 37, 37, 29, 37, 37, 28, 37, 37, 37,  6, 37, 37,
       37, 37, 20, 37, 37, 37, 18, 37, 37, 31, 31, 37, 37, 37, 37, 37, 37,
       37, 37, 37,  4, 37,  6, 37, 37,  5, 37, 37, 37, 37,  6,  6, 37, 28,
       37, 37, 37, 37, 37

In [18]:
pr = predict.predict(X[:1])
pr = K.ctc_decode(pr, [271])[0][0]
pr

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


<tf.Tensor 'SparseToDense:0' shape=(1, 43) dtype=int64>

In [22]:
pred_tr = pr.eval(session=K.get_session())

In [20]:
Y[:1]

array([[20,  3, 31, 28,  4, 29, 20, 28, 34, 31, 21,  8, 34,  0,  6, 20,
        34,  8, 28, 15, 24, 33, 14, 29, 23,  8, 10, 20,  8, 34, 20,  0,
         6, 34, 33, 20,  0, 30,  6, 20, 31, 20, 28, 28, 29, 28,  6, 20,
        10, 27,  3, 29, 30,  0, 20, 12, 29,  5, 22,  6, 20, 28,  4, 21,
         6,  4, 20, 31, 36, 20, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [34]:
import editdistance
editdistance.eval(list(pred_tr[0]), list(Y[0, :70]))

40

In [35]:
pred_tr[0]

array([20, 11,  6, 28,  4, 29, 28, 31, 36,  6, 36,  8,  5,  7, 33, 29,  1,
        8,  0,  6,  4, 33,  0,  6, 20, 31, 28, 29, 28,  6, 20, 18, 31,  4,
        6,  5,  6, 28, 15,  4, 31, 36, 20])

In [37]:
preds = predict.predict(X[:160])

In [38]:
pr = K.ctc_decode(preds, X_lens[:160])[0][0]

In [40]:
pr.eval(session=K.get_session())

array([[20, 11,  6, ..., -1, -1, -1],
       [20, 24, 22, ..., -1, -1, -1],
       [20,  6,  0, ..., -1, -1, -1],
       ...,
       [22, 31, 32, ..., -1, -1, -1],
       [20,  6, 28, ..., -1, -1, -1],
       [20, 28, 31, ..., -1, -1, -1]])

In [44]:
hypos = [list(x) for x in pr.eval(session=K.get_session())]
lens = [x.index(-1) if -1 in x else len(x) for x in hypos]
hypos = [x[:lim] for x, lim in zip(hypos, lens)]

In [52]:
gts = [list(x) for x in Y[:160]]
lens = [x.index(37) if 37 in x else len(x) for x in gts]
gts = [x[:lim] for x, lim in zip(gts, lens)]

In [56]:
[editdistance.eval(gt, hypo) / len(gt) for gt, hypo in zip(gts, hypos)]

[0.5714285714285714,
 0.5232558139534884,
 0.7580645161290323,
 0.55,
 0.6528925619834711,
 0.5851063829787234,
 0.5740740740740741,
 0.5888888888888889,
 0.5147058823529411,
 0.6086956521739131,
 0.6329113924050633,
 0.5,
 0.4461538461538462,
 0.8235294117647058,
 0.7692307692307693,
 0.7352941176470589,
 0.618421052631579,
 0.6829268292682927,
 0.5730337078651685,
 0.6265060240963856,
 0.6607142857142857,
 0.5232558139534884,
 0.5567010309278351,
 0.49019607843137253,
 0.4423076923076923,
 0.43564356435643564,
 0.4935064935064935,
 0.5606060606060606,
 0.5416666666666666,
 0.5757575757575758,
 0.5842696629213483,
 0.625,
 0.49382716049382713,
 0.5373134328358209,
 0.5520833333333334,
 0.5384615384615384,
 0.6385542168674698,
 0.5569620253164557,
 0.6162790697674418,
 0.5287356321839081,
 0.5555555555555556,
 0.5595238095238095,
 0.4457831325301205,
 0.4827586206896552,
 0.6703296703296703,
 0.5217391304347826,
 0.6197183098591549,
 0.5949367088607594,
 0.6081081081081081,
 0.57894736

In [50]:
hypos

[[20,
  11,
  6,
  28,
  4,
  29,
  28,
  31,
  36,
  6,
  36,
  8,
  5,
  7,
  33,
  29,
  1,
  8,
  0,
  6,
  4,
  33,
  0,
  6,
  20,
  31,
  28,
  29,
  28,
  6,
  20,
  18,
  31,
  4,
  6,
  5,
  6,
  28,
  15,
  4,
  31,
  36,
  20],
 [20,
  24,
  22,
  31,
  8,
  28,
  34,
  29,
  28,
  33,
  34,
  33,
  36,
  33,
  6,
  6,
  30,
  5,
  6,
  7,
  29,
  32,
  6,
  36,
  29,
  32,
  33,
  29,
  28,
  4,
  15,
  15,
  12,
  15,
  20,
  30,
  31,
  22,
  31,
  32,
  24,
  31,
  28,
  33,
  29,
  24,
  29,
  5,
  20],
 [20,
  6,
  0,
  6,
  17,
  31,
  29,
  7,
  33,
  24,
  24,
  29,
  33,
  6,
  1,
  6,
  20,
  29,
  29,
  31,
  6,
  20],
 [20,
  4,
  31,
  6,
  12,
  32,
  29,
  6,
  28,
  4,
  6,
  19,
  29,
  34,
  29,
  34,
  34,
  15,
  8,
  6,
  6,
  24,
  31,
  29,
  17,
  31,
  34,
  15,
  0,
  6,
  3,
  5,
  6,
  1,
  31,
  20,
  29,
  31,
  6,
  6,
  8,
  22,
  31,
  30,
  29,
  18,
  6,
  5,
  20],
 [20,
  31,
  29,
  6,
  29,
  6,
  28,
  4,
  29,
  31,
  36,
  29,
  6,

In [64]:
trn.evaluate(list(data), [np.zeros(len(data[0]))])

12416/12416 [==============================] - 142s 11ms/step


255.78763529197457

## 